In [1]:
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

import sys
sys.path.append("../")
import KNMIRequest as KR
import HolidayRequest as HR

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('../input/train.csv', index_col=0, parse_dates=True)
df_train = df.copy()
df_train.head()

,tripid,userid,bikeid,account,bikenumber,start_time,end_time,start_lat,start_lng,end_lat,end_lng,date
1,33838,13452,382,AT,6631000433,2019-01-01 06:46:03,2019-01-01 06:46:03,52.296065,4.787667,52.296200,4.787679,2019-01-01 06:46:00
2,33839,13452,812,AT,6631000146,2019-01-01 06:47:39,2019-01-01 06:47:39,52.295938,4.788336,52.295940,4.788557,2019-01-01 06:47:35
3,33851,13182,238,AT,6631000443,2019-01-01 14:15:36,2019-01-01 14:15:36,52.307337,4.807633,52.307336,4.807630,2019-01-01 14:15:32
4,33852,13182,238,AT,6631000443,2019-01-01 14:33:50,2019-01-01 14:33:50,52.308478,4.809970,52.308537,4.810099,2019-01-01 14:33:46
5,33867,12674,787,AI,6631000283,2019-01-01 20:11:05,2019-01-01 20:11:05,52.510340,4.725336,52.508565,4.726276,2019-01-01 20:11:01


In [3]:
df_test = pd.read_csv('../input/test.csv', index_col=0, parse_dates=True)
df_test.head()

,tripid,userid,bikeid,account,bikenumber,start_time,end_time,start_lat,start_lng,end_lat,end_lng,date
1,151899,18734,1221,AT,6631000941,2019-11-01 00:17:17,2019-11-01 00:39:04,52.285242,4.853709,52.283334,4.853159,2019-11-01 00:17:13
2,151901,14414,749,AV,6631000096,2019-11-01 01:35:09,2019-11-01 01:37:53,52.306448,4.800563,52.306460,4.800592,2019-11-01 01:35:05
3,151903,23773,758,AV,6631000273,2019-11-01 06:13:18,2019-11-01 06:16:54,52.307922,4.806427,52.307627,4.804996,2019-11-01 06:13:14
4,151905,15420,318,AT,6631000009,2019-11-01 06:39:14,2019-11-01 06:41:01,52.295016,4.790957,52.297227,4.787585,2019-11-01 06:39:10
5,151909,12504,498,AV,6631000702,2019-11-01 06:49:33,2019-11-01 06:51:17,52.294959,4.792023,52.295575,4.790326,2019-11-01 06:49:29


In [4]:
df_KNMI = KR.get_KNMI_DD(start=20190101, end=20191209, vars_=["FG", "FHX", "TN", "TX", "TG", "DR", "RH", "RHX", "VVN"])
df_KNMI.set_index("Date", inplace=True)

# renaming the KNMI columns
df_KNMI = df_KNMI.rename(columns={'FG': 'mean windspeed',
                                 'FHX': 'hightest hourly gust',
                                 'TN':  'min temp',
                                 'TX':  'max temp',
                                 'TG':  'mean temp',
                                 'DR':  'duration rain',
                                 'RH':  'sum rain',
                                 'RHX': 'highest hourly rain',
                                 'VVN': 'minimum visibility'})

ser_holidays = HR.get_holiday_range()
df_nat_hol = pd.read_csv('../input/Nationale Feestdagen.csv')
df_nat_hol['date'] = pd.to_datetime(df_nat_hol['Datum'])
df_KNMI

,Station,mean windspeed,hightest hourly gust,min temp,max temp,mean temp,duration rain,sum rain,highest hourly rain,minimum visibility
Date,,,,,,,,,,
2019-01-01,240,76,90,52,91,77,23,5,2,30
2019-01-02,240,48,70,25,71,56,0,-1,-1,62
2019-01-03,240,23,40,3,68,31,0,-1,-1,65
2019-01-04,240,45,60,10,68,54,0,-1,-1,37
2019-01-05,240,65,80,57,81,72,6,1,1,50
...,...,...,...,...,...,...,...,...,...,...
2019-12-05,240,52,90,-23,58,22,11,4,2,1
2019-12-06,240,98,120,55,100,73,95,125,31,30
2019-12-07,240,72,90,75,104,95,0,-1,-1,68


In [5]:
df = pd.read_csv('../input/train.csv', index_col=0, parse_dates=True)
df_train = df.copy()
df_train.head()

def df_prep(df, epoch):
    df['start_time'] = pd.to_datetime(df['start_time'])
    df['end_time'] = pd.to_datetime(df['end_time'])
    df['date'] = pd.to_datetime(df['date']).dt.date
    
    df = df["date"].value_counts().to_frame().reset_index().sort_values("index")
    df.rename(columns={"index" : "date", "date" : "count"}, inplace=True)

    df["date"] = pd.to_datetime(df["date"])
    df['weekday'] = df['date'].dt.day_name()
    
    df['Friday'] = (df['weekday'] == 'Friday')
    df['Weekend'] = (df['weekday'] == 'Saturday') | (df['weekday'] == 'Sunday')
    df['School holiday'] = df['date'].isin(ser_holidays)
    
    df['National holiday'] = df['date'].isin(df_nat_hol['date'].dt.date)
    
    df['Days from epoch'] = (df['date'] - epoch).dt.days + 1
    df.set_index("date", inplace=True)
    
    
    return df.join(df_KNMI).reset_index().drop(["Station", "weekday"], axis=1)

#     df['Days from epoch'] = (df['start_time'] - df['start_time'].min()).dt.days
# df_train = df_prep(df_train)
# df_train.head()

In [6]:
df_KNMI

,Station,mean windspeed,hightest hourly gust,min temp,max temp,mean temp,duration rain,sum rain,highest hourly rain,minimum visibility
Date,,,,,,,,,,
2019-01-01,240,76,90,52,91,77,23,5,2,30
2019-01-02,240,48,70,25,71,56,0,-1,-1,62
2019-01-03,240,23,40,3,68,31,0,-1,-1,65
2019-01-04,240,45,60,10,68,54,0,-1,-1,37
2019-01-05,240,65,80,57,81,72,6,1,1,50
...,...,...,...,...,...,...,...,...,...,...
2019-12-05,240,52,90,-23,58,22,11,4,2,1
2019-12-06,240,98,120,55,100,73,95,125,31,30
2019-12-07,240,72,90,75,104,95,0,-1,-1,68


In [7]:
epoch = pd.to_datetime(min(df_train["date"].min(), df_test["date"].min()))

In [8]:
df_train = df_prep(df_train, epoch)
df_train.head()

,date,count,Friday,Weekend,School holiday,National holiday,Days from epoch,mean windspeed,hightest hourly gust,min temp,max temp,mean temp,duration rain,sum rain,highest hourly rain,minimum visibility
0,2019-01-01,8,False,False,True,True,0,76,90,52,91,77,23,5,2,30
1,2019-01-02,61,False,False,True,False,1,48,70,25,71,56,0,-1,-1,62
2,2019-01-03,90,False,False,True,False,2,23,40,3,68,31,0,-1,-1,65
3,2019-01-04,64,True,False,True,False,3,45,60,10,68,54,0,-1,-1,37
4,2019-01-05,17,False,True,True,False,4,65,80,57,81,72,6,1,1,50


In [9]:
df_test = df_prep(df_test, epoch)
df_test.head()

,date,count,Friday,Weekend,School holiday,National holiday,Days from epoch,mean windspeed,hightest hourly gust,min temp,max temp,mean temp,duration rain,sum rain,highest hourly rain,minimum visibility
0,2019-11-01,180,True,False,False,False,304,46,60,15,133,79,75,38,18,22
1,2019-11-02,14,False,True,False,False,305,85,120,107,145,128,49,33,21,36
2,2019-11-03,20,False,True,False,False,306,38,80,73,124,101,46,46,19,3
3,2019-11-04,313,False,False,False,False,307,40,60,77,126,103,3,1,1,35
4,2019-11-05,499,False,False,False,False,308,25,30,72,115,96,44,36,24,56


In [10]:
df_train.to_pickle('../input/train.pkl')
df_test.to_pickle('../input/test.pkl')

## Validation data prep

In [11]:
df_validation = pd.read_csv('../input/sampleSubmission.csv', index_col=0, parse_dates=True)

df_validation.reset_index(inplace=True)
df_validation = df_validation.rename(columns={'id': 'date'})

df_validation.head()

,date,Predicted
0,2019-11-01,0
1,2019-11-02,0
2,2019-11-03,0
3,2019-11-04,0
4,2019-11-05,0


In [12]:
min_date = int(str(df_validation['date'].dt.date.min()).replace("-", ""))
max_date = int(20200131)

df_val_KNMI = KR.get_KNMI_DD(start=min_date, end=max_date, vars_=["FG", "FHX", "TN", "TX", "TG", "DR", "RH", "RHX", "VVN"])
df_val_KNMI = df_val_KNMI.rename(columns={'FG': 'mean windspeed',
                                 'FHX': 'hightest hourly gust',
                                 'TN':  'min temp',
                                 'TX':  'max temp',
                                 'TG':  'mean temp',
                                 'DR':  'duration rain',
                                 'RH':  'sum rain',
                                 'RHX': 'highest hourly rain',
                                 'VVN': 'minimum visibility'})
df_val_KNMI.head()

,Station,Date,mean windspeed,hightest hourly gust,min temp,max temp,mean temp,duration rain,sum rain,highest hourly rain,minimum visibility
0,240,2019-11-01,46,60,15,133,79,75,38,18,22
1,240,2019-11-02,85,120,107,145,128,49,33,21,36
2,240,2019-11-03,38,80,73,124,101,46,46,19,3
3,240,2019-11-04,40,60,77,126,103,3,1,1,35
4,240,2019-11-05,25,30,72,115,96,44,36,24,56


In [13]:
def data_prep_subm(df):
    df["date"] = pd.to_datetime(df["date"])
    df['weekday'] = df['date'].dt.day_name()
    
    df['Friday'] = (df['weekday'] == 'Friday')
    df['Weekend'] = (df['weekday'] == 'Saturday') | (df['weekday'] == 'Sunday')
    df['School holiday'] = df['date'].isin(ser_holidays)
    
    df['National holiday'] = df['date'].isin(df_nat_hol['date'].dt.date)
    
    df['Days from epoch'] = (df['date'] - epoch).dt.days + 1
    
    df.drop('weekday', axis=1, inplace=True)

In [14]:
data_prep_subm(df_validation)

# merge the KNMI dataset with the validation dataset
df_validation = df_validation.merge(df_val_KNMI, left_on='date', right_on='Date')
df_validation.drop('Date', axis=1, inplace=True)

df_validation.head()

,date,Predicted,Friday,Weekend,School holiday,National holiday,Days from epoch,Station,mean windspeed,hightest hourly gust,min temp,max temp,mean temp,duration rain,sum rain,highest hourly rain,minimum visibility
0,2019-11-01,0,True,False,False,False,304,240,46,60,15,133,79,75,38,18,22
1,2019-11-02,0,False,True,False,False,305,240,85,120,107,145,128,49,33,21,36
2,2019-11-03,0,False,True,False,False,306,240,38,80,73,124,101,46,46,19,3
3,2019-11-04,0,False,False,False,False,307,240,40,60,77,126,103,3,1,1,35
4,2019-11-05,0,False,False,False,False,308,240,25,30,72,115,96,44,36,24,56


In [15]:
df_validation.to_pickle('../input/validation.pkl')